In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import xml.etree.ElementTree as ET
from huggingface_hub import InferenceClient
from typing import Dict, List
import os
from groq import Groq

2025-06-05 10:59:11.039735: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 10:59:11.041742: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 10:59:11.067026: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 10:59:11.067058: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 10:59:11.067094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

## Vérifier si une page contient des tables ou pas

In [2]:
def has_table(url,type_table):
    try:
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text
    
        soup = BeautifulSoup(html_content, 'html.parser')
        external_links_header = None
        for header in soup.find_all(['h2', 'h3', 'h4', 'h5', 'h6']):
            if header.get_text().strip().lower() in ['external links', 'liens externes', 'References']:
                external_links_header = header
                break
    
        if external_links_header:
            for sibling in external_links_header.find_all_next():
                sibling.decompose()
        tables_html = soup.find_all('table')
        for table in tables_html:
            if any(c in table.get('class', []) for c in ['ambox', 'metadata', 'box-Primary_sources', 'nowraplinks' ]):
                continue
            if table.get('role') == 'presentation':
                continue
            if type_table == "table":
                if 'infobox' not in table.get('class', []):
                    return True
            elif type_table == "infobox":
                if 'infobox' in table.get('class', []):
                    return True
                
        
        return False
    except:
        pass



In [16]:
with open("liens_wiki_all.txt") as f:
    urls = f.readlines()
Links = [url.strip() for url in urls]

In [ ]:
Tables_links = []
for link in Links:
    if has_table(link, "table"):
        Tables_links.append(link)

In [2]:
with open("links_wiki_with_tables.txt") as f:
    urls = f.readlines()
Tables_links = [url.strip() for url in urls]

In [3]:
len(Tables_links)

354

## Liens ves les pages Wikipédia

### 1. Tous liens ves les pages contenant tables

#### 1.1) Au moins table de type "table"

In [3]:
with open("links_wiki_with_tables.txt") as f:
    urls = f.readlines()
Tables_links = [url.strip() for url in urls]

In [4]:
len(set(Tables_links))

354

#### 1.2) Au moins table de type "infobox"

In [1]:
#Infobox_links = []
#for link in Links:
#    if has_table(link, "infobox"):
#        Infobox_links.append(link)

In [2]:
#len(set(Infobox_links))

### 2. Liens vers les pages que nous avons annotés

In [6]:
folder_path = "./tables/"
csv_files = [file.split(".")[0] for file in os.listdir(folder_path) if file.endswith('.csv')]
links = []
for file in csv_files :
    link = f"https://en.wikipedia.org/wiki/{file}"
    links.append(link)

In [7]:
len(links)

99

### Nouvelle liens (hors (2))

In [8]:
new_links = [link for link in Tables_links if link not in links]

In [14]:
len(set(new_links))

266

In [10]:
new_links = list(set(new_links))

# Extract Wikipedia Tables

In [19]:

def extract_wikipedia_tables(url):
    response = requests.get(url)
    response.raise_for_status()
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')

    external_links_header = None
    for header in soup.find_all(['h2', 'h3', 'h4', 'h5', 'h6']):
        if header.get_text().strip().lower() in ['external links', 'liens externes']:
            external_links_header = header
            break

    if external_links_header:
        for sibling in external_links_header.find_all_next():
            sibling.decompose()

    tables_html = soup.find_all('table')
    infobox_tables_html = soup.find_all('table', class_='infobox')

    dataframes = []

    for table in infobox_tables_html:
        table_str = str(table)
        df_list = pd.read_html(io.StringIO(table_str))
        if df_list:
            df = df_list[0]
            dataframes.append(('infobox', df))

    for table in tables_html:
        if 'infobox' not in table.get('class', []):
            print("ok")
            table_str = str(table)
            df_list = pd.read_html(io.StringIO(table_str))
            if df_list:
                df = df_list[0]
                dataframes.append(('table', df))

    return dataframes


In [20]:
pages_tables = {}
for link in new_links:
    try:
        tables = extract_wikipedia_tables(link)
        pages_tables[link]=tables
    except:
        pass

ok


In [20]:
prem

'https://en.wikipedia.org/wiki/Blue-Tongue_Films'

In [21]:
len(pages_tables)

0

In [14]:
prem = next(iter(pages_tables))

In [29]:
type(pages_tables[prem][0][1])

pandas.core.frame.DataFrame

# Extract content from wikipedia pages

In [8]:
links = ['https://en.wikipedia.org/wiki/Access_Bank_Botswana',
 'https://en.wikipedia.org/wiki/Afrasia_Bank_Zimbabwe_Limited',
 'https://en.wikipedia.org/wiki/Altitude_Film_Distribution']

In [11]:
links = [link for link in pages_tables]


In [12]:
pages_contents= {}
links_have_problems = []
for k in range(len(links)):
    try:
        page_contents = {}
        soup = BeautifulSoup(urlopen(links[k]).read(), 'html.parser')
        headers = soup.find_all(re.compile(r'h[1-6]', re.I))
    
        for header in headers:
            header_level = header.name
            title_span = header.span["class"] if header.span and "class" in header.span.attrs else None
            title = header.text.strip()
            next_node = header.find_next()
            paragraphs = []
    
            while next_node and (not next_node.name or not next_node.name.startswith('h')):
                if next_node.name == 'p':
                    text = next_node.get_text(strip=True)
                    if text:
                        paragraphs.append(text)
                next_node = next_node.find_next()
    
            if paragraphs:
                #print(f"{header_level}: {title}")
                page_contents[f"{header_level}: {title}"] = {
                    f"P{i+1}": para for i, para in enumerate(paragraphs) if re.search(r'[a-zA-Z]', para)
                }
        pages_contents[links[k]] = page_contents
    except :
        links_have_problems.append(links[k])

In [13]:
len(pages_contents)

261

In [14]:
links_have_problems

['https://en.wikipedia.org/wiki/Øresundslinjen']

In [15]:
del pages_tables["https://en.wikipedia.org/wiki/Øresundslinjen"]

In [16]:
len(pages_tables)

261

# Transform tables to texts

In [34]:
prem = next(iter(pages_tables))

In [40]:
pages_tables['https://en.wikipedia.org/wiki/CNET_Video'][0][1]

,0,1
0,Company type,Online media content provider
1,Industry,Mass media
2,Founded,2005
3,Headquarters,"San Francisco, California, U.S."
4,Products,Video podcasts
5,Website,www.cnet.com/videos/


In [55]:
page_table_test = {}
page_table_test ['https://en.wikipedia.org/wiki/CNET_Video'] = pages_tables['https://en.wikipedia.org/wiki/CNET_Video']

In [17]:

def dataframe_to_xml(df, root_name='root', row_name='row'):
    root = ET.Element(root_name)
    for _, row in df.iterrows():
        element = ET.SubElement(root, row_name)
        for col in df.columns:
            child = ET.SubElement(element, str(col))
            child.text = str(row[col])
    return ET.tostring(root, encoding='unicode')

pages_tables_as_xlm = {}
for link in pages_tables:
    contents_as_xlm = []
    for table_as_tupe in pages_tables[link]:
        xml_data = dataframe_to_xml(table_as_tupe[1], root_name='Table', row_name='Ligne')
        content_as_xlm = (table_as_tupe[0], xml_data)
        contents_as_xlm.append(content_as_xlm)
    pages_tables_as_xlm[link] = contents_as_xlm


In [22]:
# Tranfsormer chaque table to dictionnaire
pages_tables_as_dic = {}
for link in pages_tables:
    contents_as_dict = []
    for table_as_tupe in pages_tables[link]:
        dict_data = table_as_tupe[1].to_dict(orient='records')
        content_as_dict = (table_as_tupe[0], dict_data)
        contents_as_dict.append(content_as_dict)
    pages_tables_as_dic[link] = contents_as_dict


In [23]:
pages_tables_as_dic['https://en.wikipedia.org/wiki/CNET_Video']

[('infobox',
  [{0: 'Company type', 1: 'Online media content provider'},
   {0: 'Industry', 1: 'Mass media'},
   {0: 'Founded', 1: '2005'},
   {0: 'Headquarters', 1: 'San Francisco, California, U.S.'},
   {0: 'Products', 1: 'Video podcasts'},
   {0: 'Website', 1: 'www.cnet.com/videos/'}]),
 ('table',
  [{'Year': 2012,
    'Nominee / work': '"\'CNET Video"\'',
    'Award': "Webby Award for Technology People's Voice",
    'Result': 'Won'},
   {'Year': 2010,
    'Nominee / work': '"\'CNET Video"\'',
    'Award': "Webby Award for Technology People's Voice",
    'Result': 'Won'},
   {'Year': 2010,
    'Nominee / work': '"\'CNET Video"\'',
    'Award': 'Webby Award for Technology',
    'Result': 'Won'},
   {'Year': 2009,
    'Nominee / work': '"\'CNET Video"\'',
    'Award': "Webby Award for Technology People's Voice",
    'Result': 'Won'}])]

## Use LLMs To transform Table to text

In [79]:
os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"] ="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"
#os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"]= "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"
#os.environ["GROQ_API_KEY"] = "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"
#os.environ["GROQ_API_KEY"]= "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"

In [80]:
client = Groq()
DEFAULT_MODEL = "llama-3.3-70b-versatile"
#DEFAULT_MODEL = "deepseek-r1-distill-llama-70b"
def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = 0.1,
    top_p: float = 0.2,
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content
        

def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.1,
    top_p: float = 0.2,
) -> str:
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    #print(f'==============\n{prompt}\n==============')
    response = completion(prompt, model)
    #print(response, end='\n\n')
    return response

In [65]:
prompt = """Carefully analyze the table and provide a complete description of all rows and columns with only correct sentences (without lists). Make sure to include Every row in your explanation without skipping any details."""

In [ ]:
demo_instructions=[]
for i in range(len(xlm_tables_v)):
    #demo_instructions.append(f"""What is the 'relation' between "{e1}" and "{e2}" in the context of the input sentence. Please the output format should be '{e1},{e2}: relation'. \nSentence: {demo_inputs[i]}\nOptions: acquired_by, brand_of,client_of,collaboration,competitor_of, merged_with, product_or_service_of,regulated_by,shareholder_of,subsidiary_of,traded_on,undefined""")
    demo_instructions.append(f""" {prompt}\n\n
    Table : {xlm_tables_v[i]}\n\n
    Output: """)

In [49]:
pages_tables_as_dict_59_100 = dict(list(pages_tables_as_dic.items())[59:100])

In [88]:
len(contents_with_LLM)

3

In [90]:
pages_tables_with_LLM_description = {}
cpt = 0
for link in pages_tables_as_dict_59_100:
    cpt+= 1 
    contents_with_LLM = []
    for element in pages_tables_as_dict_59_100[link]:
        instruction = f"{prompt}\nTable: {element[1]}\n\nOutput: "
        reponse = complete_and_print(instruction)
        resultat = (element[0], reponse)
        contents_with_LLM.append(resultat)
    pages_tables_with_LLM_description[link] = contents_with_LLM
    if cpt % 10 == 0:
        print(cpt)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jnxcjtswekfs9z5jjzbg7hf2` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99807, Requested 17777. Please try again in 4h13m12.491999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [32]:
resultats = []


In [46]:
resultats.append(pages_tables_with_LLM_description)

In [82]:
pages_tables_with_LLM_description = {}
for element in resultats:
    for key in element:
        pages_tables_with_LLM_description[key] = element[key]

In [83]:
len(pages_tables_with_LLM_description)

59

In [84]:
prem = next(iter(pages_tables_with_LLM_description))

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

#model_name = "deepseek-ai/deepseek-llm-67b-base"
model_name = "/projects/melodi/mettaleb/FinGPT/fingpt/FinGPT_Benchmark/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

text = "An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyboardInterrupt: 

In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)


2.6.0+cu124
12.4


In [9]:
pwd

'/projects/melodi/mettaleb/Annotation'

# Similarity between table and paragraphs

In [ ]:
import copy
model = SentenceTransformer('all-mpnet-base-v2')
pages_contents_scores = copy.deepcopy(pages_contents)
for i in range(len(csv_files)):
    encoding1 = model.encode(generated_texts[i])
    try:
        for section in pages_contents[csv_files[i]]:
            for paragraphe in pages_contents[csv_files[i]][section]:
                encoding2 = model.encode(pages_contents[csv_files[i]][section][paragraphe])
                similarity = np.dot(encoding1, encoding2) / (np.linalg.norm(encoding1) * np.linalg.norm(encoding2))
                pages_contents_scores[csv_files[i]][section][paragraphe] = similarity
    except:
        pass